In [11]:
import csv
import time
import pandas as pd
from random import randint
from urllib.request import urlopen

from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup

from telnetlib import EC
from pynput.keyboard import Key, Controller
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait

In [12]:
urlLinkedIn_path = "E:/CMT_GLOBAL_GROUP/INTERN/newBotLinkedIn/recourses/url_linkedin.csv"
df = pd.read_csv(urlLinkedIn_path)
df.head()

,LinkedIn_Link,Status_Connect,Message,Status Message
0,https://www.linkedin.com/in/mrivet/,Not Connected,hi,Message not sent
1,https://www.linkedin.com/in/jaimeleverton/,Connected,hi,Message sent
2,https://www.linkedin.com/in/gabriellowenberg-7...,Pending,hi,Message not sent
3,https://www.linkedin.com/in/cameron-b-hurst-bb...,Not Connected,hi,Message not sent
4,https://www.linkedin.com/in/jrbeattie/,Pending,hi,Message not sent


In [13]:
# tài khoản đăng nhập LinkedIn cần cần connect
userName = 'example@gmail.com'
password = 'password'
LinkedIn_link = "https://www.linkedin.com"
LinkedIn_login = "https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin"

# auto connect linkedin

- mở LinkedIn
- đăng nhập
- xác thực bot (nếu có)
- mở link cần connect
- nhấn vào connect

In [26]:
# connect LinkedIn function v1
# đoạn code này sẽ:
# mở LinkedIn
# đăng nhập
# từ các đường link trong csv thì mở nó lên và nhấn kết nối
# do là đẵ đăng nhập trước rồi, nên việc mở các url_linkedIn sẽ dễ dàng kết nối hơn!

driver = Chrome()

# driver.get(LinkedIn_link)
# mở link rồi thử đăng nhập với tài khoản được cung cấp ở trên
try:
    driver.get(LinkedIn_login)
    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')

    # điền tài khoản
    userName_input = driver.find_element(By.ID, "username")
    userName_input.send_keys(userName)

    # điền mật khẩu
    password_input = driver.find_element(By.ID, "password")
    password_input.send_keys(password)

    # click đăng nhập
    login_tag = bs.find_all('div', {'class': 'login__form_action_container'})
    print(login_tag)
    button_element = driver.find_element(By.CLASS_NAME, 'btn__primary--large')
    button_element.click()
    time.sleep(randint(5, 10))
    # đến đây là đăng nhập thành công!


    # thực hiện tìm các liên hệ và connect
    link_ex = df.iloc[0, 0]
    driver.get(link_ex)

    # đến đây sẽ có 2 trường hợp: hoặc là đã follow hoặc là chưa follow
    try:
        followButon = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/button")
        button_text1 = followButon.get_attribute("aria-label")
        if "Invite" in button_text1:
            try:               
                # kiểm tra trạng thái kết nối và nhấn kết nối 
                checkStatus = followButon.get_attribute("aria-label")
                print("This person is not connected , try to click connect..........")
                followButon.click()
                print("click connect!")
                time.sleep(2)
                connectButonNew = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[3]/button[2]")
                connectButonNew.click()
                time.sleep(2)

                # if "Pending" in checkStatus:
                #     print("Invite sent !" + df['LinkedIn_Link'][value])
                #     writeCvs("Invite sent", value + 1)
                #     writeCvsMessage("Message not sent", value + 1)
                
            except Exception as eee:
                print(f'Some error at (3) {eee}')

        elif "Follow" in button_text1:
            try:
                connectButtonNew1 = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/button")
                connectButtonNew1.click()
                time.sleep(1)

                connectButonNew2 = driver.find_element(By.XPATH,"/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/div/div/ul/li[3]/div")
                connectButonNew2.click()
                connectButonNew3 = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[3]/button[2]")
                connectButonNew3.click()
                time.sleep(2)
                followButon1 = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/button")
                test = followButon1.get_attribute("aria-label")

                # if("Pending" in test):
                #     print('value: ' + str(value))
                #     writeCvs("Invite sent", value + 1)
                # else:
                #     writeCvs("Not connected", value + 1)

            except Exception as eeee:
                print(f'Some error at (4) {eeee}')

        elif "Pending" in button_text1:
            # print("Pending!" + data['LinkedIn_Link'][value])
            # writeCvs("Pending", value + 1)
            # print("write in csv!")
            # writeCvsMessage("Message not sent", value + 1)
            pass

    except Exception as ee:
        print(f'Some error at (2) {ee}')
        # nếu đã kết nối rồi thì sẽ gửi tin nhắn
        try:
            checkStatus = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/button")
            checkStatus.click()
            time.sleep(1)

            checkStatus = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/div/div/ul/li[7]/div")
            checkStatus2 = checkStatus.get_attribute("aria-label")

            if "Remove" in checkStatus2:
                # print("Connected one!" + data['LinkedIn_Link'][value])
                # writeCvs("Connected", value + 1)
                # print("write in csv!")
                
                try:
                    sendMessageButton = driver.find_element(By.XPATH,"/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[1]/button")
                    print("Message buton found")
                    sendMessageButton.click()
                    time.sleep(1)

                    # print("value is : " + str(value) )
                    # driver.find_element(By.XPATH,"/html/body/div[5]/div[4]/aside[1]/div[2]/div[1]/div[2]/div/form/div[2]/div/div[1]/div[1]").send_keys(data['Message'][value])
                    
                    driver.find_element(By.XPATH, "/html/body/div[5]/div[4]/aside[1]/div[2]/div[1]/div[2]/div/form/footer/div[2]/div[1]/button").click()
                    time.sleep(4)
                    print("click")
                    # writeCvsMessage("Message sent", value + 1)

                except Exception as e5:
                    try:
                        # driver.find_element(By.XPATH, "//div[contains(@class, 'msg-form__contenteditable')]").send_keys(data['Message'][value])
                        time.sleep(2)

                        driver.find_element(By.XPATH, "/html/body/div[5]/div[4]/aside[1]/div[2]/div[1]/div[2]/div/form/footer/div[2]/div[1]/button").click()
                        time.sleep(4)

                        driver.find_element(By.XPATH, "/html/body/div[5]/div[4]/aside[1]/div[2]/div[1]/header/div[4]/button[3]").click()
                        time.sleep(0.5)
                        # writeCvsMessage("Message sent", value + 1)
                        
                    except Exception as e6:
                        print(f'Some error at (6) {e6}')
                
        except Exception as eeeee:
            print(f'Some error at (5) {eeeee}')

except Exception as e:
    print(f'Some error at (1) {e}')

# driver.quit()

[<div class="login__form_action_container">
<!-- --> <button aria-label="Sign in" class="btn__primary--large from__button--floating" data-litms-control-urn="login-submit" type="submit">
              Sign in
          </button>
</div>]
This person is not connected , try to click connect..........
click connect!


# auto connection linkedin v2

In [ ]:
# duyệt qua tất cả các profile sẽ được kết nối và thực hiện kết nối

driver = Chrome()

driver.get(LinkedIn_link)

try:
    driver.get(LinkedIn_login)
    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')

    # điền tài khoản
    userName_input = driver.find_element(By.ID, "username")
    userName_input.send_keys(userName)

    # điền mật khẩu
    password_input = driver.find_element(By.ID, "password")
    password_input.send_keys(password)

    # click đăng nhập
    login_tag = bs.find_all('div', {'class': 'login__form_action_container'})
    print(login_tag)
    button_element = driver.find_element(By.CLASS_NAME, 'btn__primary--large')
    button_element.click()
    time.sleep(randint(5, 10))
    # đến đây là đăng nhập thành công!


    # thực hiện tìm các liên hệ và connect
    # link_ex = df.iloc[0, 0]
    # driver.get(link_ex)
    for idx in range(0, len(df)):
        driver.get(df.iloc[idx, 0])
        # đến đây sẽ có 2 trường hợp: hoặc là đã follow hoặc là chưa follow
        try:
            followButon = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/button")
            button_text1 = followButon.get_attribute("aria-label")
            if "Invite" in button_text1:
                try:               
                    # kiểm tra trạng thái kết nối và nhấn kết nối 
                    checkStatus = followButon.get_attribute("aria-label")
                    print("This person is not connected , try to click connect..........")
                    followButon.click()
                    print("click connect!")
                    time.sleep(2)
                    connectButonNew = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[3]/button[2]")
                    connectButonNew.click()
                    time.sleep(2)

                    # if "Pending" in checkStatus:
                    #     print("Invite sent !" + df['LinkedIn_Link'][value])
                    #     writeCvs("Invite sent", value + 1)
                    #     writeCvsMessage("Message not sent", value + 1)
                    
                except Exception as eee:
                    print(f'Some error at {eee}')

            elif "Follow" in button_text1:
                try:
                    connectButtonNew1 = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/button")
                    connectButtonNew1.click()
                    time.sleep(1)

                    connectButonNew2 = driver.find_element(By.XPATH,"/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/div/div/ul/li[3]/div")
                    connectButonNew2.click()
                    connectButonNew3 = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[3]/button[2]")
                    connectButonNew3.click()
                    time.sleep(2)
                    followButon1 = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/button")
                    test = followButon1.get_attribute("aria-label")

                    # if("Pending" in test):
                    #     print('value: ' + str(value))
                    #     writeCvs("Invite sent", value + 1)
                    # else:
                    #     writeCvs("Not connected", value + 1)

                except Exception as eeee:
                    pass

            elif "Pending" in button_text1:
                # print("Pending!" + data['LinkedIn_Link'][value])
                # writeCvs("Pending", value + 1)
                # print("write in csv!")
                # writeCvsMessage("Message not sent", value + 1)
                pass

        except Exception as ee:
            print(f'Some error at {ee}')
            # nếu đã kết nối rồi thì sẽ gửi tin nhắn
            try:
                checkStatus = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/button")
                checkStatus.click()
                time.sleep(1)

                checkStatus = driver.find_element(By.XPATH, "/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/div/div/ul/li[7]/div")
                checkStatus2 = checkStatus.get_attribute("aria-label")

                if "Remove" in checkStatus2:
                    # print("Connected one!" + data['LinkedIn_Link'][value])
                    # writeCvs("Connected", value + 1)
                    # print("write in csv!")
                    
                    try:
                        sendMessageButton = driver.find_element(By.XPATH,"/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[1]/button")
                        print("Message buton found")
                        sendMessageButton.click()
                        time.sleep(1)

                        # print("value is : " + str(value) )
                        # driver.find_element(By.XPATH,"/html/body/div[5]/div[4]/aside[1]/div[2]/div[1]/div[2]/div/form/div[2]/div/div[1]/div[1]").send_keys(data['Message'][value])
                        
                        driver.find_element(By.XPATH, "/html/body/div[5]/div[4]/aside[1]/div[2]/div[1]/div[2]/div/form/footer/div[2]/div[1]/button").click()
                        time.sleep(4)
                        print("click")
                        # writeCvsMessage("Message sent", value + 1)

                    except Exception as e5:
                        try:
                            # driver.find_element(By.XPATH, "//div[contains(@class, 'msg-form__contenteditable')]").send_keys(data['Message'][value])
                            time.sleep(2)

                            driver.find_element(By.XPATH, "/html/body/div[5]/div[4]/aside[1]/div[2]/div[1]/div[2]/div/form/footer/div[2]/div[1]/button").click()
                            time.sleep(4)

                            driver.find_element(By.XPATH, "/html/body/div[5]/div[4]/aside[1]/div[2]/div[1]/header/div[4]/button[3]").click()
                            time.sleep(0.5)
                            # writeCvsMessage("Message sent", value + 1)
                            
                        except Exception as e6:
                            pass
                    
            except Exception as eeeee:
                continue

except Exception as e:
    print(f'Some error at {e}')
driver.quit()